In [14]:
import os

os.environ["OMP_NUM_THREADS"] = "1"

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import corner
import batman
import emcee
import time
from multiprocessing import Pool
from scipy import optimize
from IPython.display import display, Math

# import sys
# sys.path.insert(0, r'/home/rk19047/PycharmProjects/MSci2022_TESS/packages')
# from unit_conversion import radius_in_stellar_radii, semi_major_axis_in_stellar_radii

ImportError: cannot import name 'radius_in_stellar_radii' from 'unit_conversion' (/home/rk19047/PycharmProjects/MSci2022_TESS/packages/unit_conversion.py)

In [16]:
np.random.seed(100)

In [17]:
#System
#http://exoplanets.org/detail/WASP-17b

# Semi-Major Axis[au]	0.04994 ± 0.00083
# Orbital Period[day]	3.7354330 +7.5×10-6/-7.7×10-6
# Orbital Eccentricity	0
# Orbit Inclination[deg]	86.63 +0.39/-0.45
# Argument of Periastron[deg]	90

# Time of Periastron[jd]	2454559.18096 +0.00025/-0.00021
# Transit Parameters
# Planetary Radius[rjupiter]	1.93 +0.052/-0.1
# Impact Parameter	0.400 +0.043/-0.04

# Stellar Properties
# Star Name	WASP-17
# Binary Flag	✗
# Mass of Star[msun]	1.190 ± 0.03
# Radius of Star[rsun]	1.200 +0.081/-0.08
# [Fe/H]	-0.250 ± 0.09
# Teff[k]	6550 ± 100
# Density of star[g/cm^3]	0.67 +0.16/-0.13
# log10(g)	4.20 ± 0.2
# Vsin(i)[km/s]	9.0 ± 1.5
# Gamma[km/s]	-49

In [18]:
true_R_star = 1.200 #R_sun
true_R_planet = 1.93 #R_J
true_semi_major_axis = 0.04994 #AU


params = batman.TransitParams()
params.t0 = 0.                                          #time of inferior conjunction
params.per = 3.7354330                                        #orbital period (days)
params.rp = radius_in_stellar_radii(true_R_planet, true_R_star)                #planet radius (in units of stellar radii)
params.a = semi_major_axis_in_stellar_radii(true_semi_major_axis, true_R_star) #semi-major axis (in units of stellar radii))
params.inc = 89.94                                       #orbital inclination (in degrees)
params.ecc = 0.0                                         #eccentricity
params.w = 90.                                          #longitude of periastron (in degrees)
params.limb_dark = "quadratic"                          #limb darkening model
params.u = [0.21768792857990628,
            0.2836292883489928]                             #limb darkening coefficients [u1, u2, u3, u4]

t = np.linspace(0, 30, 21600)                      #times at which to calculate light curve
m = batman.TransitModel(params, t)                      #initializes model
true_flux = m.light_curve(params)

NameError: name 'radius_in_stellar_radii' is not defined